### IMPORTS

In [2]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [3]:
from Carga_Descarga import *
from CRM_Central import *

In [4]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### GOOGLE SHEETS

In [72]:
# Descargo los filtros de Campañas
fc = descarga('1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw','Filtrar')
# Mando a mayusculas
fc[fc.columns[0]] = fc[fc.columns[0]].str.upper()
fc = fc[fc.columns[0]].tolist()

Descarga Correcta!


### FUNCIONES

In [71]:
def filtrar_cam(i):
    val = 'No'
    for j in fc:
        if j in i:
            val = 'Si'
            break
    return val        

### QUERIES

In [97]:
q_gen = '''WITH orders_table AS (
    SELECT o.user.id AS user,
           o.order_id AS order_id,
           o.registered_date AS fecha,
           CASE WHEN o.is_acquisition = 1 THEN True ELSE False END AS is_acquisition,
           cn.country_name AS country
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE o.registered_date >= DATE('2021-01-01')
          AND o.order_status = 'CONFIRMED'),
    users_table AS (
    SELECT u.user_id AS user,
           cn.country_name AS country
    FROM `peya-data-origins-pro.cl_glovo_migration.glovo_customer_migrated` AS gcm
    INNER JOIN `peya-bi-tools-pro.il_core.dim_user` AS u ON gcm.peya_id = u.user_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON u.country_id = cn.country_id
    WHERE cn.country_name IN ('Perú','Ecuador','Honduras','Costa Rica','Guatemala')),
    cross_table AS (
    SELECT DISTINCT FORMAT_DATE('%Y-%m',d.date) AS month,
           cn.country_name AS country
    FROM `peya-bi-tools-pro.il_core.dim_date` AS d
    CROSS JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn
    WHERE cn.country_name IN ('Perú','Ecuador','Honduras','Costa Rica','Guatemala')
          AND d.date BETWEEN DATE('2021-01-01') AND DATE('2021-07-31'))
SELECT ct.month AS Month,
       ct.country AS Country,
       COUNT(DISTINCT ut.user) AS Total_Glovo_Users,
       COUNT(DISTINCT CASE WHEN ot.is_acquisition THEN ot.user ELSE NULL END) AS Glovo_Acq,
       COUNT(DISTINCT ot.order_id) AS Confirmed_Glovo_Orders
FROM cross_table AS ct
LEFT JOIN users_table AS ut ON ct.country = ut.country
LEFT JOIN orders_table AS ot ON ut.user = ot.user AND ut.country = ot.country AND ct.month = FORMAT_DATE('%Y-%m',ot.fecha)
GROUP BY 1,2
ORDER BY 2,1 ASC'''

q_coh = '''WITH acq_table AS (
    SELECT u.user_id AS user,
           cn.country_name AS country,
           o.registered_date AS fecha
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    INNER JOIN `peya-bi-tools-pro.il_core.dim_user` AS u ON o.user.id = u.user_id AND o.country_id = u.country_id
    INNER JOIN `peya-data-origins-pro.cl_glovo_migration.glovo_customer_migrated` AS gcm ON u.user_id = gcm.peya_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON u.country_id = cn.country_id
    WHERE o.registered_date >= DATE('2021-01-01')
          AND cn.country_name IN ('Perú','Ecuador','Honduras','Costa Rica','Guatemala')
          AND o.is_acquisition = 1
          AND o.order_status = 'CONFIRMED'),
    cohorts_table AS (
    SELECT acq.user AS user,
           acq.country AS country,
           acq.fecha AS fecha,
           DATE_DIFF(o.registered_date,acq.fecha,MONTH) AS cohort_month,
           IFNULL(COUNT(DISTINCT o.order_id),0) AS orders
    FROM acq_table AS acq
    LEFT JOIN `peya-bi-tools-pro.il_core.fact_orders` AS o ON acq.user = o.user.id
    WHERE o.registered_date >= DATE('2021-01-01')
          AND o.registered_date BETWEEN DATE_ADD(acq.fecha,INTERVAL 1 DAY) AND CURRENT_DATE()
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2,3,o.registered_date),
    acq_final_table AS (
    SELECT FORMAT_DATE('%Y-%m',acq.fecha) AS fecha,
           acq.country AS country,
           COUNT(DISTINCT acq.user) AS users
    FROM acq_table AS acq
    GROUP BY 1,2)
SELECT FORMAT_DATE('%Y-%m',ct.fecha) AS Month,
       ct.country AS Country,
       IFNULL(aft.users,0) AS Glovo_Acq,
       COUNT(DISTINCT CASE WHEN ct.cohort_month = 1 THEN ct.user ELSE NULL END) AS Active_Glovo_M1,
       COUNT(DISTINCT CASE WHEN ct.cohort_month = 2 THEN ct.user ELSE NULL END) AS Active_Glovo_M2,
       COUNT(DISTINCT CASE WHEN ct.cohort_month = 3 THEN ct.user ELSE NULL END) AS Active_Glovo_M3
FROM cohorts_table AS ct
LEFT JOIN acq_final_table AS aft ON FORMAT_DATE('%Y-%m', ct.fecha) = aft.fecha AND ct.country = aft.country
GROUP BY 1,2,3
ORDER BY 2,1'''

q_acq = '''SELECT FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       cn.country_name AS Country,
       COUNT(DISTINCT o.order_id) AS Confirmed_Totales,
       COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS Acq_Totales
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
WHERE o.registered_date >= DATE('2021-01-01')
      AND o.order_status = 'CONFIRMED'
      AND cn.country_name IN ('Perú','Ecuador','Honduras','Costa Rica','Guatemala')
GROUP BY 1,2
ORDER BY 2,1'''

q_tc = '''SELECT FORMAT_DATE('%Y-%m',vd.date) AS Month,
       vd.country_name AS Country,
       vd.campaign_name AS Campaign,
       CASE WHEN vd.payment_shopper = 1 THEN 'Si' ELSE 'No' END AS Shopper,
       SUM(vd.coupons_redeemed_confirmed) AS Redeemed_Confirmed,
       SUM(vd.coupon_orders_acq_confirmed) AS Acquisitions,
       SUM(vd.coupon_used_amount_confirmed / ce.rate_us) AS Amount_USD
FROM `peya-growth-and-onboarding.automated_tables_reports.peya_vouchers_daiy` AS vd
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON vd.country_name = cn.country_name
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(vd.date,MONTH) = ce.currency_exchange_date
WHERE vd.date >= DATE('2021-01-01')
      AND vd.country_name IN ('Perú','Ecuador','Honduras','Costa Rica','Guatemala')
GROUP BY 1,2,3,4
ORDER BY 2,1'''

q_tcg = '''WITH coupons_table AS (
    SELECT cn.country_name AS country,
           tc.talon_campaign_id AS campaign_id,
           tc.coupon_id AS coupon_id,
           o.registered_date AS fecha,
           COUNT(DISTINCT o.order_id) AS redeemed,
           COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS redeemed_confirmed,
           COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS acquisitions,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' AND bi.payment_mode = 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS used_amount_confirmed_usd,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' AND bi.payment_mode != 'TOTAL_AMOUNT' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS used_amount_confirmed_usd_shopper
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    INNER JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
    INNER JOIN `peya-bi-tools-pro.il_core.dim_user` AS u ON o.user.id = u.user_id AND o.country_id = u.country_id
    INNER JOIN `peya-data-origins-pro.cl_glovo_migration.glovo_customer_migrated` AS gcm ON u.user_id = gcm.peya_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON u.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
    WHERE o.registered_date >= DATE('2021-01-01')
          AND cn.country_name IN ('Perú','Ecuador','Honduras','Costa Rica','Guatemala')
          AND DATE(tc.coupon_created_at) >= DATE('2020-01-01')
    GROUP BY 1,2,3,4),
    coupons_final AS (
    SELECT ct.country AS country,
           FORMAT_DATE('%Y-%m',ct.fecha) AS month,
           ct.campaign_id AS campaign_id,
           SUM(ct.redeemed) AS redeemed,
           SUM(ct.redeemed_confirmed) AS redeemed_confirmed,
           SUM(ct.acquisitions) AS acq,
           SUM(ct.used_amount_confirmed_usd) AS used_amount_confirmed_usd,
           SUM(ct.used_amount_confirmed_usd_shopper) AS used_amount_confirmed_usd_shopper
    FROM coupons_table AS ct
    GROUP BY 1,2,3)
SELECT cf.country AS Country,
       cf.month AS Month,
       tca.talon_campaign_name AS Campaign,
       SUM(cf.redeemed) AS Redeemed,
       SUM(cf.redeemed_confirmed) AS Redeemed_Confirmed,
       SUM(cf.acq) AS Acq,
       SUM(cf.used_amount_confirmed_usd) AS Used_Amount_Confirmed_USD,
       SUM(cf.used_amount_confirmed_usd_shopper) AS Used_Amount_Confirmed_USD_Shopper
FROM coupons_final AS cf 
LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON cf.campaign_id = tca.talon_campaign_id
GROUP BY 1,2,3'''

In [20]:
# Descargo la data
bq_gen = pd.io.gbq.read_gbq(q_gen, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 110.05rows/s]


In [21]:
bq_coh = pd.io.gbq.read_gbq(q_coh, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 74.69rows/s]


In [22]:
bq_acq = pd.io.gbq.read_gbq(q_acq, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 83.02rows/s]


In [51]:
bq_tc = pd.io.gbq.read_gbq(q_tc, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 1135/1135 [00:00<00:00, 2321.65rows/s]


In [98]:
bq_tcg = pd.io.gbq.read_gbq(q_tcg, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 1017/1017 [00:00<00:00, 1826.88rows/s]


In [108]:
# Copio las bases
gen = bq_gen.copy()
coh = bq_coh.copy()
acq = bq_acq.copy()
tc = bq_tc.copy()
tcg = bq_tcg.copy()

### TRABAJO

In [109]:
# Formateo Gen
cols_st = ['Month','Country']
cols_fl = [i for i in gen.columns if i not in cols_st]
gen[cols_st] = gen[cols_st].apply(lambda x: x.astype(str).str.upper())
gen[cols_fl] = gen[cols_fl].astype(int)
gen['Month'] = pd.to_datetime(gen['Month'], format='%Y-%m').dt.strftime('%Y-%m')
# Formateo Gen
cols_st = ['Month','Country']
cols_fl = [i for i in coh.columns if i not in cols_st]
coh[cols_st] = coh[cols_st].apply(lambda x: x.astype(str).str.upper())
coh[cols_fl] = coh[cols_fl].astype(int)
coh['Month'] = pd.to_datetime(coh['Month'], format='%Y-%m').dt.strftime('%Y-%m')
# Formateo Acq
cols_st = ['Month','Country']
cols_fl = [i for i in acq.columns if i not in cols_st]
acq[cols_st] = acq[cols_st].apply(lambda x: x.astype(str).str.upper())
acq[cols_fl] = acq[cols_fl].astype(int)
acq['Month'] = pd.to_datetime(acq['Month'], format='%Y-%m').dt.strftime('%Y-%m')
# Formateo TC
cols_st = ['Month','Country','Campaign','Shopper']
cols_fl = [i for i in tc.columns if i not in cols_st]
tc[cols_st] = tc[cols_st].apply(lambda x: x.astype(str).str.upper())
tc[cols_fl] = tc[cols_fl].astype(float)
tc['Month'] = pd.to_datetime(tc['Month'], format='%Y-%m').dt.strftime('%Y-%m')
tc['Campaign'] = tc['Campaign'].str.replace(' ', '')
# Formateo TCG
cols_st = ['Month','Country','Campaign']
cols_fl = [i for i in tcg.columns if i not in cols_st]
tcg[cols_st] = tcg[cols_st].apply(lambda x: x.astype(str).str.upper())
tcg[cols_fl] = tcg[cols_fl].astype(float)
tcg['Month'] = pd.to_datetime(tcg['Month'], format='%Y-%m').dt.strftime('%Y-%m')
tcg['Campaign'] = tcg['Campaign'].str.replace(' ', '')

In [110]:
# Hago un merge entre las tablas
cols = ['Month','Country','Active_Glovo_M1','Active_Glovo_M2','Active_Glovo_M3']
final = gen.merge(coh[cols],on=['Month','Country'],how='left')
final = final.merge(acq,on=['Month','Country'],how='left')
final.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [111]:
# Creo las columnas para los segmentos
tc['Benefit'] = tc['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
tc['Type'] = tc['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
tc['Segment'] = tc['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
tc['Level'] = tc['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
tc['Campaña'] = tc['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
tc['Automated'] = tc['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
tc['Budget'] = tc['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
tc['Trial'] = tc['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
# Coloco el IVA
tc = func_iva(tc)
# Marco las campañas a filtrar
tc['Filtrar'] = tc['Campaign'].apply(filtrar_cam)
tc = tc[tc['Filtrar'] == 'No'].copy()
# Saco el IVA
used = 'Amount_USD'
tc.loc[tc['Shopper'] == 'NO','Amount_Final'] = tc[used] / (1 + tc['IVA']) * tc['Extra']
tc.loc[tc['Shopper'] == 'SI','Amount_Final'] = tc[used]

In [107]:
tcg.columns

Index(['Month', 'Country', 'Campaign', 'Shopper', 'Redeemed_Confirmed',
       'Acquisitions', 'Amount_USD', 'Benefit', 'Type', 'Segment', 'Level',
       'Campaña', 'Automated', 'Budget', 'Trial', 'IVA_x', 'Extra_x',
       'Filtrar', 'Amount_Final', 'IVA_y', 'Extra_y'],
      dtype='object')

In [112]:
# Creo las columnas para los segmentos
tcg['Benefit'] = tcg['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
tcg['Type'] = tcg['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
tcg['Segment'] = tcg['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
tcg['Level'] = tcg['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
tcg['Campaña'] = tcg['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
tcg['Automated'] = tcg['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
tcg['Budget'] = tcg['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
tcg['Trial'] = tcg['Campaign'].apply(lambda x: filtros(x,'TRIAL'))
# Coloco el IVA
tcg = func_iva(tcg)
# Marco las campañas a filtrar
tcg['Filtrar'] = tcg['Campaign'].apply(filtrar_cam)
tcg = tcg[tcg['Filtrar'] == 'No'].copy()
# Saco el IVA
used = 'Used_Amount_Confirmed_USD'
used_iva = 'Used_Amount_Confirmed_USD_Shopper'
tcg['Amount_Final'] = tcg[used_iva] / (1 + tcg['IVA']) * tcg['Extra'] + tcg[used]

In [93]:
# Creo una PT con los gastos Generales
index = ['Month','Country','Budget','Type','Segment']
values = ['Redeemed_Confirmed','Acquisitions','Amount_Final']
final_tc = tc.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index().copy()
final_tc.sort_values(['Country','Month','Budget','Type','Segment'],inplace=True)

In [115]:
# Creo una PT con los gastos de Glovo
index = ['Month','Country','Budget','Type','Segment']
values = ['Redeemed_Confirmed','Acq','Amount_Final']
final_tcg = tcg.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index().copy()
final_tcg.sort_values(['Country','Month','Budget','Type','Segment'],inplace=True)

### CARGA

In [116]:
carga(final_tcg,'15ahfbUgf3HQzvJemHcq0-F2WBnad9V_NkcIxKmJWVrY','Gasto Paises CRM Glovo')

Carga Correcta!


In [96]:
carga(final_tc,'15ahfbUgf3HQzvJemHcq0-F2WBnad9V_NkcIxKmJWVrY','Gasto Paises CRM')

Carga Correcta!


In [36]:
carga(final,'15ahfbUgf3HQzvJemHcq0-F2WBnad9V_NkcIxKmJWVrY','Crudo')

Carga Correcta!
